# Project Title
### Data Engineering Capstone Project

#### Project Summary

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

### Step 1: Scope the Project and Gather Data

#### Scope 
The project will investigate I94 immigration data, world temperature data, and US infographic data and try to provide insights.  

In this project, the plan is to assess the data and formulate possible statistics that can be derived.  The data is cleaned and catered towards deriving useful data.  A list of the possible immigration statistics is as follows:
- Immigration by city
- Age of immigrants 
- Gender of immigrants
- Reason for travel and visa type
- Length of stay 
- Mode of travel
- Temperature of cities immigrated to 
- Country of origin

The tools that will be used for the project are as follows:
- Python / Pandas

#### Describe and Gather Data 

The data that is going to be used is as follows:
- **I94 Immigration Data:** The data is provided by the US National Tourism and Trade Office.  The site that provides the data can be found [HERE](https://travel.trade.gov/research/reports/i94/historical/2016.html)
- **World Temperature Data:** This data comes from Kaggle and more information can be found [HERE](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data)
- **US City Demographic Data:** This data comes from OpenSoft and more information can be found [HERE](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/)
- **Airport Code Table:** This is a table of airport codes with the corresponding cities they are located in and more information can be found [HERE](https://datahub.io/core/airport-codes#data)
- **I94 SAS Labels Descriptions:**  This data is derived from a provided SAS file and several tables are created to supplement the above data:
    - Country code conversions
    - Port code conversions
    - Mode of travel conversions
    - State code conversions
    - Visa type conversions

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Do all imports and installs here
import pandas as pd
import pyspark
import calendar

pd.set_option('display.max_rows', 30)

### **I94 Immigration Data**

In [ ]:
i94_df = pd.read_sas('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat', 'sas7bdat', encoding='ISO-8859-1')
print(len(i94_df))

In [ ]:
i94_df.head(10)

In [ ]:
# Display data types

display(i94_df.dtypes)

#### **Data Analysis**

Upon inspection of the data, I am making the following assumptions:
- According to the SAS Label Descriptions that correspond to this data, it states that many of the columns are not used by the CIC, so they will be omitted from the data model
- The arrdate and depdate columns will need to be converted to readable date types
- The isnum column contains many nulls and will likely not be used as a dependable 
- The first column is not named, but assuming it is a type of record ID.  
- Many of the data types are floats when they could be integers (year, month, etc)

#### **Data Cleaning**
- Columns that are not going to be used will be omitted from the data model
- The cicid column will be used as the primary key as a duplicate check has indicated that there are only unique values

#### **immigration_data_sample.csv**
| Column                   | Description                                | Used in Dim Table |
| -----------------------  | ------------------------------------------ | :---------------: |
| cicid (float)            | Immigrant identifier                       |        X          |
| i94yr (float)            | 4 digit year                               |        X          |
| i94mon (float)           | Numeric month                              |        X          |
| i94cit (float)           | Resident birth city                        |        X          |
| i94res (text)            | Resident origin res                        |        X          |
| i94port (float)          | Arrival port                               |        X          |
| arrdate (float)          | Arrival date in US                         |        X          |
| i94mode (float)          | Mode of travel                             |        X          |
| i94addr (text)           | Address state                              |        X          |
| depdate (float)          | Date departing from US                     |        X          |
| i94bir (float)           | Age of respondent                          |        X          |
| i94visa (float)          | Visa code (visa use)                       |        X          |
| count (float)            | Used for summarizing statistics            |                   |
| dtadfile (int)           | Data dded to files (not used)              |                   |
| visapost (text)          | State where visa was issued (not used)     |                   |
| occup (text)             | Occupation in US (not used)                |                   |
| entdepa (char)           | Arrival flag (not used)                    |                   |
| entdepd (char)           | Departure flag (not used)                  |                   |
| entdepu (text)           | Update flag (not used)                     |                   |
| matflag (char)           | Match flag                                 |                   |
| biryear (float)          | 4 digit year of birth                      |                   |
| dtaddto (int)            | Data admitted to US (not used)             |                   |
| gender (char)            | Sex of non-immigrant                       |        X          |
| insnum (int)              | INS Number                                 |                   |
| airline (text)           | Airline used to arrive in US               |        X          |
| admnum (int)             | Admission number                           |                   |
| fltno (text)             | Flight number                              |                   |
| visatype (text)          | Class of admission for temp state          |        X          |

In [ ]:
# Drop columns that will not be used in the data model (see data model)
i94_clean = i94_df.drop(columns=['count', 'dtadfile', 'visapost', 'entdepa', 'entdepd', 'entdepu', 'matflag', 'biryear', 'dtaddto', 'insnum', 'admnum', 'fltno', 'occup'])

i94_clean.head(10)

In [ ]:
# Check if first column has duplicate records
check = i94_clean['cicid'].duplicated().any()

print(check)

In [ ]:
# Remove records with null values
i94_clean = i94_clean.dropna(how='any',axis=0)

print(len(i94_clean))

In [ ]:
# Convert fields with float to int
i94_clean['cicid'] = i94_clean['cicid'].astype(int)
i94_clean['i94yr'] = i94_clean['i94yr'].astype(int)
i94_clean['i94mon'] = i94_clean['i94mon'].astype(int)
i94_clean['i94yr'] = i94_clean['i94yr'].astype(int)
i94_clean['i94cit'] = i94_clean['i94cit'].astype(int)
i94_clean['i94res'] = i94_clean['i94res'].astype(int)
i94_clean['i94bir'] = i94_clean['i94bir'].astype(int)
i94_clean['i94visa'] = i94_clean['i94visa'].astype(int)
i94_clean['i94mode'] = i94_clean['i94mode'].astype(int)

In [ ]:
# Convert arrdate and depdate from float to date

i94_clean['arrdate'] = pd.to_datetime(i94_clean['arrdate']).dt.date
i94_clean['depdate'] = pd.to_datetime(i94_clean['depdate']).dt.date

i94_clean.head()


### **World Temperature Data**

In [ ]:
world_temp_df = pd.read_csv('../../data2/GlobalLandTemperaturesByCity.csv')
print(len(world_temp_df))

In [ ]:
world_temp_df.head(10)

In [ ]:
# Display data types
display(world_temp_df.dtypes)

#### **Data Analysis**

Upon inspection of the data, I am making the following assumptions:
- The data has average temps for each month from 1743-2013 from several cities and countries around the world. 
- Temperatures can be derived for both the immigrant's country of origin as well as for the United States

#### **Data Cleaning**
- Columns that are not going to be used will be omitted from the data model
- The data will be averaged for each country by month - a new column for month will be created
- Rows that have null temperature data will be dropped before aggregation

### **GlobalLandTemperatureByCity.csv**
| Column                             | Description                      | Used in Dim Table |
| ---------------------------------  | -------------------------------- | :---------------: |
| dt (date)                          | Date temp taken                  |        X          |
| avg temperature (float)            | Average temp                     |        X          |
| avg temperature uncertainty (float)| Accounts for margin of error     |        X          |
| city (text)                        | City                             |        X          |
| country (text)                     | Country                          |        X          |
| latitude (text)                    | Latitude                         |                   |
| longitude (text)                   | Longitude                        |                   |

In [ ]:
# Drop columns that will not be used in the data model (see data model)
world_temp_clean = world_temp_df.drop(columns=['Latitude', 'Longitude'])

world_temp_clean.head(10)

In [ ]:
# Remove records with null values
world_temp_clean = world_temp_clean.dropna(how='any',axis=0)

print(len(world_temp_clean))

In [ ]:
# Create column for month
world_temp_clean['Month'] = pd.DatetimeIndex(world_temp_clean['dt']).month
world_temp_clean.head()

In [ ]:
# Calculate average temps for each city and country by month
world_temp_agg = world_temp_clean.groupby(['City','Country', 'Month' ])['AverageTemperature', 'AverageTemperatureUncertainty'].mean().reset_index()

world_temp_agg.head(30)

### **US City Demographic Data**

In [ ]:
us_demo_df = pd.read_csv('us-cities-demographics.csv', sep = ';')
print(len(us_demo_df))

In [ ]:
us_demo_df.head(10)

In [ ]:
# Display data types
display(us_demo_df.dtypes)

#### **Data Analysis**

Upon inspection of the data, I am making the following assumptions:
- The data displays basic demographics for various cities in the US

#### **Data Cleaning**
- Columns that are not going to be used will be omitted from the data model
- Columns with float types will be converted to int as appropriate
- Rows that have null data will be dropped

### **us-cities-demographics.csv**
| Column                   | Description                                | Used in Dim Table |
| -----------------------  | ------------------------------------------ | :---------------: |
| city (text)              | City in US                                 |        X          |
| state (text)             | State in US                                |        X          |
| median age (float)       | Median age of population                   |        X          |
| male population (int)    | Number of males in population              |        X          |
| female population (text) | Number of females in population            |        X          |
| total population (int)   | Total people in population                 |        X          |
| number of veterans (int) | Veterans in population                     |                   |
| foreign-born (int)       | Number of immigrants                       |        X          |
| avg household size (int) | Average size of household                  |        X          |
| state code (text)        | State abbreviation                         |        X          |
| race (text)              | Race                                       |                   |
| count (int)              | Count of number of race in population      |                   |

In [ ]:
# Drop columns that will not be used in the data model (see data model)
us_demo_clean = us_demo_df.drop(columns=['Number of Veterans', 'Race', 'Count'])

us_demo_clean.head(10)

In [ ]:
# Remove records with null values
us_demo_clean = us_demo_clean.dropna(how='any',axis=0)

print(len(us_demo_clean))

In [ ]:
# Convert male and female populations from float to int

us_demo_clean['Male Population'] = us_demo_clean['Male Population'].astype(int)
us_demo_clean['Female Population'] = us_demo_clean['Female Population'].astype(int)

us_demo_clean.head()

### **Airport Code Table**

In [ ]:
airport_df = pd.read_csv('airport-codes_csv.csv')
print(len(airport_df))

In [ ]:
airport_df.head(10)

In [ ]:
# Display data types in the columns
display(airport_df.dtypes)

#### **Data Analysis**

Upon inspection of the data, I am making the following assumptions:
- It looks like parts of the ident codes will match with port in the immigration table, but not exactly
- Given the ideas for analysis presented in the scope, I will only consider airports within the US
- Need a way to derive the state

#### **Data Cleaning**
- Columns that are not going to be used will be omitted from the data model
- Columns with float types will be converted to int as appropriate
- Rows that have null data will be dropped

### **airport-codes_csv.csv**
| Column                | Description                                   | Used in Dim Table |
| --------------------  | --------------------------------------------- | :---------------: |
| ident (text)          | Airport identification code                   |        X          |
| type (text)           | Type of airport                               |        X          |
| name (text)           | Name of airport                               |        X          |
| elevation_ft (int)    | Airport elevation                             |                   |
| continent (text)      | Continent                                     |                   |
| iso_country (text)    | Country                                       |        X          |
| iso_region (text)     | Country and region                            |        X          |
| municipality (text)   | City                                          |        X          |
| gps_code (text)       | GPS Code                                      |                   |
| iata_code (text)      | Airport location identifier                   |                   |
| local_code (text)     | Airport code                                  |                   |
| coordinates (text)    | Coordinates                                   |                   |

In [ ]:
# Drop columns that will not be used in the data model (see data model)
airport_clean = airport_df.drop(columns=['elevation_ft', 'continent', 'gps_code', 'iata_code', 'local_code', 'coordinates'])

airport_clean.head(10)

In [ ]:
# Remove rows that are not US
airport_clean = airport_clean[airport_clean.iso_country == 'US']
print(len(airport_clean))

In [ ]:
# Create state column
states = airport_clean['iso_region'].str.split('-', n = 1, expand = True)
airport_clean['state'] = states[1]
airport_clean = airport_clean.drop(columns=['iso_region'])

airport_clean.head(10)

### **I94_SAS_Labels_Descriptions**

Additional csv files were created using the data available in the I94_SAS_Labels_Descriptions.SAS file.  

In [ ]:
# I94 Travel Modes
travel_modes_df = pd.read_csv('i94mode.csv')
travel_modes_df.head(10)

In [ ]:
# I94 Visa Types
visa_types_df = pd.read_csv('visa_type.csv')
visa_types_df.head(10)

In [ ]:
# I94 Ports
ports_df = pd.read_csv('i94prtl.csv')
ports_df.head(10)

In [ ]:
# I94 State Codes
states_df = pd.read_csv('i94addrl.csv')
states_df = states_df.drop(columns=['Unnamed: 2'])
states_df.head(10)

In [ ]:
# I94 Countries
countries_df = pd.read_csv('i94cntyl.csv')
countries_df = countries_df.drop(columns=['Unnamed: 2'])
countries_df.head(10)

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
The model was designed using a star schema because it would be more performant than the snowflake schema as there are less joins to have to work with.

![Data Model](data_model.png)

#### 3.2 Mapping Out Data Pipelines

The steps that will be taken for the data pipeline are as follows:
- Setting up a connection to a local Sparkify database
- Renaming the columns of the cleaned data frames as appropriate 
- Loading the data frames into the database
- Adding city column to the fact table to help support the star schema
- Perform data quality checks

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Import modules 
import os
import psycopg2
from sql_queries import *

In [ ]:
# Create database
conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=student password=student")
conn.set_session(autocommit=True)
cur = conn.cursor()

# Create sparkify database with UTF8 Encoding
cur.execute("drop database if exists sparkifydb")
cur.execute("create database sparkifydb with encoding 'utf8' template template0")

# close connection to default database
conn.close()  

In [ ]:
# Create connection to database
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [ ]:
# Drop existing tables using the drop_table_queries list
for query in drop_table_queries:
    cur.execute(query)
    conn.commit()

In [ ]:
# Create tables using the create_table_queries list
for query in create_table_queries:
    cur.execute(query)
    conn.commit()

In [ ]:
# Populate airport_codes
airport_codes_data = airport_clean.values
airport_codes_data = airport_codes_data.tolist()

try:
    cur.executemany(airport_codes_table_insert, airport_codes_data)
except psycopg2.Error as e:
    print("Error: Inserting Rows Into Airport Codes Table")
    print(e)

In [ ]:
# Populate global_temps
global_temps_data = world_temp_agg.values
global_temps_data = global_temps_data.tolist()

try:
    cur.executemany(global_temps_table_insert, global_temps_data)
except psycopg2.Error as e:
    print("Error: Inserting Rows Into Global Temps Table")
    print(e)

In [ ]:
# Populate us_cities_demographics
us_demo_data = us_demo_clean.values
us_demo_data = us_demo_data.tolist()

try:
    cur.executemany(us_cities_demographics_table_insert, us_demo_data)
except psycopg2.Error as e:
    print("Error: Inserting Rows Into US Cities Demographics Table")
    print(e)

In [ ]:
# Populate countries
countries_data = countries_df.values
countries_data = countries_data.tolist()

try:
    cur.executemany(countries_table_insert, countries_data)
except psycopg2.Error as e:
    print("Error: Inserting Rows Into US Countries Table")
    print(e)

In [ ]:
# Populate ports
ports_data = ports_df.values
ports_data = ports_data.tolist()

try:
    cur.executemany(ports_table_insert, ports_data)
except psycopg2.Error as e:
    print("Error: Inserting Rows Into Ports Table")
    print(e)

In [ ]:
# Populate states
states_data = states_df.values
states_data = states_data.tolist()

try:
    cur.executemany(states_table_insert, states_data)
except psycopg2.Error as e:
    print("Error: Inserting Rows Into States Table")
    print(e)

In [ ]:
# Populate travel_modes
travel_mode_data = travel_modes_df.values
travel_mode_data = travel_mode_data.tolist()

try:
    cur.executemany(travel_modes_table_insert, travel_mode_data)
except psycopg2.Error as e:
    print("Error: Inserting Rows Into Travel Modes Table")
    print(e)

In [ ]:
# Populate visa_use
visa_use_data = visa_types_df.values
visa_use_data = visa_use_data.tolist()

try:
    cur.executemany(visa_use_table_insert, visa_use_data)
except psycopg2.Error as e:
    print("Error: Inserting Rows Into Visa Use Table")
    print(e)

In [ ]:
# Create city column to augment immigrant table

# Join i94_clean and ports_df - inner join so we have city info for every record
immigrants = pd.merge(i94_clean, ports_df, how='inner', left_on = 'i94port', right_on = 'code')
immigrants = immigrants.drop(columns=['code', 'port'])

immigrants.head(10)

In [ ]:
# Populate immigrant table
immigrants_data = immigrants.values
immigrants_data = immigrants_data.tolist()

try:
    cur.executemany(immigrant_table_insert, immigrants)
except psycopg2.Error as e:
    print("Error: Inserting Rows Into Immigrants Table")
    print(e)

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
def GetTableLength(table):
    try:
        cur.execute(f'select count(*) from {table}')
        airport_table_records = cur.fetchone()
        return airport_table_records[0]
    except psycopg2.Error as e:
        print('Error: Querying airport_codes Table')
        print(e)

In [ ]:
def CheckLengths(df_length, table_length):
    if df_length == table_length:
        return True
    else:
        return False

In [ ]:
# Checks to ensure that the number of expected records are inserted into the database tables
passed = []
failed = []

# airport_codes
airport_df_length = len(airport_clean)
airport_table_records = GetTableLength('airport_codes')

match = CheckLengths(airport_df_length, airport_table_records)

if match:
    passed.append('airport_codes')
else:
    failed.append('airport_codes')
    
# global_temps
global_temps_length = len(world_temp_agg)
global_temps_table_records = GetTableLength('global_temps')

match = CheckLengths(global_temps_length, global_temps_table_records)

if match:
    passed.append('global_temps')
else:
    failed.append('global_temps')
    
# us_cities_demographics
us_demo_length = len(us_demo_clean)
us_demo_table_records = GetTableLength('us_cities_demographics')

match = CheckLengths(us_demo_length, us_demo_table_records)

if match:
    passed.append('us_cities_demographics')
else:
    failed.append('us_cities_demographics')
    
# countries
countries_length = len(countries_df)
countries_table_records = GetTableLength('countries')

match = CheckLengths(countries_length, countries_table_records)

if match:
    passed.append('countries')
else:
    failed.append('countries')
    
# ports
ports_length = len(ports_df)
ports_table_records = GetTableLength('ports')

match = CheckLengths(ports_length, ports_table_records)

if match:
    passed.append('ports')
else:
    failed.append('ports')
    
# states
states_length = len(states_df)
states_table_records = GetTableLength('states')

match = CheckLengths(states_length, states_table_records)

if match:
    passed.append('states')
else:
    failed.append('states')
    
# travel_modes
travel_modes_length = len(travel_modes_df)
travel_modes_table_records = GetTableLength('travel_modes')

match = CheckLengths(travel_modes_length, travel_modes_table_records)

if match:
    passed.append('travel_modes')
else:
    failed.append('travel_modes')
    
# visa_use
visa_use_length = len(visa_types_df)
visa_use_table_records = GetTableLength('visa_use')

match = CheckLengths(visa_use_length, visa_use_table_records)

if match:
    passed.append('visa_use')
else:
    failed.append('visa_use')
    
# immgrant
immigrants_length = len(immigrants)
immigrants_table_records = GetTableLength('immigrants')

match = CheckLengths(immigrants_length, immigrants_table_records)

if match:
    passed.append('immigrants')
else:
    failed.append('immigrants')

print('Tests Passed:')
print(passed)
print('Tests Failed:')
print(failed)

#### 4.3 Data dictionary 

### *Dimension Tables*

#### **airport_codes**
| Column                | Description                                   | 
| --------------------  | --------------------------------------------- | 
| ident (text)          | Airport identification code                   |
| type (text)           | Type of airport                               | 
| name (text)           | Name of airport                               |
| country (text)        | Country                                       |
| state (text)          | State                                         | 
| city (text)           | City                                          | 

&nbsp;

#### **global_temps**
| Column                             | Description                      |
| ---------------------------------  | -------------------------------- |
| city (text)                        | City                             |
| country (text)                     | Country                          |
| month (int)                        | Month                            |
| avg_temperature (float)            | Average temp                     |
| avg_temperature_uncertainty (float)| Accounts for margin of error     |

&nbsp;

#### **us_cities_demographics**
| Column                   | Description                                |
| -----------------------  | ------------------------------------------ |
| city (text)              | City in US                                 |
| state (text)             | State in US                                |
| median_age (float)       | Median age of population                   |
| male_population (int)    | Number of males in population              |
| female_population (text) | Number of females in population            |
| total_population (int)   | Total people in population                 |
| foreign_born (int)       | Number of immigrants                       |
| avg_household_size (int) | Average size of household                  |
| state_code (text)        | State abbreviation                         |

&nbsp;

#### **countries**
| Column                             | Description                      |
| ---------------------------------  | -------------------------------- |
| code (text)                        | Country code                     |
| country (text)                     | Country                          |

&nbsp;

#### **ports**
| Column                             | Description                      |
| ---------------------------------  | -------------------------------- |
| code (text)                        | Port code                        |
| port (text)                        | Port                             |
| city (text)                        | City                             |
| state (text)                       | State                            |

&nbsp;

#### **states**
| Column                             | Description                      |
| ---------------------------------  | -------------------------------- |
| code (text)                        | State code                       |
| state (text)                       | State                            |

&nbsp;

#### **travel_modes**
| Column                             | Description                      |
| ---------------------------------  | -------------------------------- |
| code (int)                         | Travel code                      |
| mode (text)                        | Travel type                      |

&nbsp;

#### **visa_use**
| Column                             | Description                      |
| ---------------------------------  | -------------------------------- |
| code (int)                         | Visa code                        |
| visa (text)                        | Visa type                        |

&nbsp;

### *Fact Table*

#### **immigrant**
| Column                   | Description                                |
| -----------------------  | ------------------------------------------ |
| cicid (int)              | Immigrant identifier                       |
| year (int)               | 4 digit year                               |
| month (int)              | Numeric month                              |
| birth_res (text)         | Resident birth                             |
| origin_res (text)        | Resident origin                            |
| port (int)               | Arrival port                               |
| arrdate (date)           | Arrival date in US                         |
| travel_mode (int)        | Mode of travel                             |
| res_state (text)         | Resident state                             |
| depdate (date)           | Date departing from US                     |
| age (int)                | Age of respondent                          |
| visa_use (int)           | Visa code (visa use)                       |
| gender (char)            | Sex of non-immigrant                       |
| airline (text)           | Airline used to arrive in US               |
| visa_type (text)         | Class of admission for temp state          |
| port_city  (text)        | Arrival city                               |
| port_state  (text)       | Arrival state                              |

#### Step 5: Complete Project Write Up

#### **Overview**

In this project, we investigated I94 immigration data, world temperature data, and US infographic data and try to provide insights.  

#### **Tools**

The primary tool that I utilized for this exercise was python, specifically, the pandas library.  While I could have just as easily used pyspark, for data exploration, especially with the smaller data sets, I just tend to prefer using pandas.  It allowed me the ability to do everything I needed to do:  explore data, clean data, and interact with databases.

#### **Data Updates**

Since the data use in this project, there was little consideration for data updates.  Processing data updates would require running all of the cells in this notebook again.  In a production-type pipeline, however, it would be possible to keep the data updated in a more automated fashion.

#### **Scenarios**

**The data is increased by 100x**

With a larger dataset, I would leverage more AWS technologies to help to process the data.  I would likely use pyspark and employee a technology that would allow processing to be more performant, such as an AWS Lambda, and AWS EC2, or an AWS EMR Cluster.

**The data populates a dashboard that must be updated on a daily basis by 7am every day**

In this scenario, I would set up a pipeline that would allow for automated updates such as AirFlow, Glue, or a Lambda with a trigger.  As part of the pipeline, I would use a persistent database such as RDS or Redshift if cost is not a factor.  My first experiment, however, would likely be a Glue Job writing to Athena tables and connecting the dashboard to Athena.

**The database needs to be accessed by 100+ people**

In the case where several users need to access the database, I would try to find a persistent and performant database that is also cost effective.  I would take one of two routes as far as providing data availability:  spin up a serverless RDS and use IAM to control the users who are looking to access the data, or store the data in Athena and use workgroups and user tokens to allow users to access.